# 演示使用向量数据库 chromadb 来检索。

https://docs.trychroma.com/getting-started

In [3]:
import os
import socket
import socks

socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 10808)
socket.socket = socks.socksocket

In [4]:
#!pip install chromadb

In [55]:
import chromadb

chroma_client = chromadb.Client()
chroma_client.delete_collection(name='my_collection')
collection = chroma_client.create_collection(name="my_collection")

collection.add(
    documents=["4月1日是日本能登半岛地震发生满三个月。日本媒体3月31日报道，受灾地石川县目前仍有超过8000人处于避难状态，数千户家庭停水，大批受损房屋无人处置，重建家园遥遥无期。", 
               "法国内政部长热拉尔德·达尔马宁3月30日下午在首都巴黎附近的圣但尼地区说，在过去10天左右的缉毒行动中，警方查缴了240万欧元，缴获150公斤毒品，其中主要是可卡因和海洛因。仅在巴黎及其东北部的塞纳-圣但尼省，警方就拘留了300多人。",
               "4月3日6时56分，我国在西昌卫星发射中心使用长征二号丁运载火箭，成功将遥感四十二号01星发射升空，卫星顺利进入预定轨道，发射任务获得圆满成功。新华社发（杨熙 摄）新华社西昌4月3日电（李国利、崔婉莹）4月3日6时56分，我国在西昌卫星发射中心使用长征二号丁运载火箭，成功将遥感四十二号01星发射升空，卫星顺利进入预定轨道，发射任务获得圆满成功"
            ]
    ,
    metadatas=[{"source": "my_source"}, {"source": "my_source"}, {"source": "my_source"}],
    ids=["id1", "id2", "id3"]
)


In [56]:
results = collection.query(
    query_texts=["地震"],
    n_results=2
)
print(results['documents'][0]) 

['4月1日是日本能登半岛地震发生满三个月。日本媒体3月31日报道，受灾地石川县目前仍有超过8000人处于避难状态，数千户家庭停水，大批受损房屋无人处置，重建家园遥遥无期。', '法国内政部长热拉尔德·达尔马宁3月30日下午在首都巴黎附近的圣但尼地区说，在过去10天左右的缉毒行动中，警方查缴了240万欧元，缴获150公斤毒品，其中主要是可卡因和海洛因。仅在巴黎及其东北部的塞纳-圣但尼省，警方就拘留了300多人。']


In [58]:
from openai import OpenAI

client = OpenAI()

# 构造聊天记录
messages=[
    {'role': 'system', 'content': '请你根据用户提供的资料作答，不要做任何的预设，确保你的回答是有依据来源的。'},
    {'role': 'user', 'content': '2024年中国遥感42号卫星发射使用何种火箭？'}
 ]

data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
  temperature=0.5
)

message = data.choices[0].message.content
print(message)


根据2024年中国遥感42号卫星的发射计划，该卫星将使用长征四号丙火箭（CZ-4C）进行发射。


In [59]:

question = '2024年中国遥感42号卫星发射使用何种火箭？'

info = collection.query(query_texts=[question],n_results=1)['documents'][0][0]

content = question+'基于以下事实：\n'+info

print(content)

2024年中国遥感42号卫星发射使用何种火箭？基于以下事实：
4月3日6时56分，我国在西昌卫星发射中心使用长征二号丁运载火箭，成功将遥感四十二号01星发射升空，卫星顺利进入预定轨道，发射任务获得圆满成功。新华社发（杨熙 摄）新华社西昌4月3日电（李国利、崔婉莹）4月3日6时56分，我国在西昌卫星发射中心使用长征二号丁运载火箭，成功将遥感四十二号01星发射升空，卫星顺利进入预定轨道，发射任务获得圆满成功


In [60]:

# 构造聊天记录
messages=[
    {'role': 'system', 'content': '请你根据用户提供的资料作答，不要做任何的预设，确保你的回答是有依据来源的。'},
    {'role': 'user', 'content': content}
 ]

data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
  temperature=0
)

message = data.choices[0].message.content
print(message)

根据提供的资料，2024年中国遥感42号卫星发射使用的是长征二号丁运载火箭。
